# Image Augmentation

In [ ]:
# Image augmentation
import cv2
import random
import os

In [ ]:
# Visualization
from matplotlib import pyplot as plt

In [ ]:
# Training
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

from sklearn.utils import class_weight

import numpy as np
import model
import time

In [ ]:
SIZE = 512
channel = 1

In [ ]:
# Image augmentation
def data_gen(img_folder, mask_folder, batch_size):
    c = 0
    n = os.listdir(img_folder) #List of training images
    random.shuffle(n)

    while (True):
        img = np.zeros((batch_size, SIZE, SIZE, channel)).astype('float')
        mask = np.zeros((batch_size, SIZE, SIZE, 1)).astype('float')

        for i in range(c, c+batch_size): #initially from 0 to 16, c = 0. 

            # If training on RGB
#             train_img = cv2.imread(img_folder+'/'+n[i])/255.       
#             train_img = cv2.resize(train_img, (SIZE, SIZE))# Read an image from folder and resize
            
            # If training on grayscale images 
            train_img = cv2.imread(img_folder+'/'+n[i], cv2.IMREAD_GRAYSCALE)/255.
            train_img = cv2.resize(train_img, (SIZE, SIZE))# Read an image from folder and resize
            train_img = train_img.reshape(SIZE, SIZE, channel) # Add extra dimension for parity with train_img size [512 * 512 * 3]
    
            img[i-c] = train_img #add to array - img[0], img[1], and so on.

            train_mask = cv2.imread(mask_folder+'/'+n[i], cv2.IMREAD_GRAYSCALE)/255.
            train_mask = cv2.resize(train_mask, (SIZE, SIZE))
            train_mask = train_mask.reshape(SIZE, SIZE, 1) # Add extra dimension for parity with train_img size [512 * 512 * 3]

            mask[i-c] = train_mask

        c+=batch_size
        if(c+batch_size>=len(os.listdir(img_folder))):
            c=0
            random.shuffle(n)
          
        yield img, mask


train_frame_path = 'train_frames/train'
train_mask_path = 'train_masks/train'

val_frame_path = 'val_frames/val'
val_mask_path = 'val_masks/val'

# Create generator objects
train_gen = data_gen(train_frame_path,train_mask_path, batch_size = 4)
val_gen = data_gen(val_frame_path,val_mask_path, batch_size = 4)



In [ ]:
# Check to see if mask is saved correctly for training set
image_batch, mask_batch = next(train_gen)

r = random.randint(0, len(image_batch)-1)

fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)
ax = fig.add_subplot(1, 2, 1)
ax.imshow(np.reshape(image_batch[r], (SIZE, SIZE)))
ax = fig.add_subplot(1, 2, 2)
ax.imshow(np.reshape(mask_batch[r], (SIZE, SIZE)), cmap="gray")

In [ ]:
# Check to see if mask is saved correctly for validation set
# image_batch, mask_batch = next(val_gen)

# r = random.randint(0, len(image_batch)-1)

# fig = plt.figure()
# fig.subplots_adjust(hspace=0.4, wspace=0.4)
# ax = fig.add_subplot(1, 2, 1)
# ax.imshow(image_batch[r].reshape(SIZE, SIZE, 3))
# ax = fig.add_subplot(1, 2, 2)
# ax.imshow(np.reshape(mask_batch[r], (SIZE, SIZE)), cmap="gray")

# Training

In [ ]:
m = model.unet(input_size= (SIZE, SIZE, channel))

In [ ]:
import loss

In [ ]:
batch_size = 5
NO_OF_TRAINING_IMAGES = len(os.listdir('train_frames/train/'))
NO_OF_VAL_IMAGES = len(os.listdir('val_frames/val/'))

NO_OF_EPOCHS = 50

BATCH_SIZE = batch_size

weights_path = 'weights/weights_00.h5'

opt = Adam(lr=1E-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

m.compile(loss=loss.dice_coef_loss,
          optimizer=opt,
          metrics=['acc', 'mae'])

checkpoint = ModelCheckpoint(filepath=weights_path, monitor='val_loss', 
                             verbose=1, save_best_only=True, save_weights_only=True)

csv_logger = CSVLogger('./log.out', append=True, separator=';')

earlystopping = EarlyStopping(monitor = 'val_loss', verbose = 1,
                              min_delta = 1e-4, patience = 2, mode = 'auto')

callbacks_list = [checkpoint, csv_logger, earlystopping]

# class_weights = {0: 1.,
#                  1: 197.}

In [ ]:
start_time = time.time()
results = m.fit_generator(train_gen, epochs=NO_OF_EPOCHS, 
                          steps_per_epoch = (NO_OF_TRAINING_IMAGES//BATCH_SIZE),
                          validation_data=val_gen, 
                          validation_steps=(NO_OF_VAL_IMAGES//BATCH_SIZE), 
                          callbacks=callbacks_list)
time_passed = time.time() - start_time
# print('Ellapsed time: {}'.format(hms_string(time_passed))
m.save('Model_00.h5')
# print(str(time_passed))

In [ ]:
print ("It took {} hours to execute this".format((time_passed) / 3600.))

# Evaluate model on test dataset

In [ ]:
# Create testing dataset
test_frame_path = 'test_frames'
test_mask_path  = 'test_masks'

test_gen = data_gen(test_frame_path, test_mask_path, batch_size = batch_size)

In [ ]:
# Loading test data
image_batch, mask_batch = next(test_gen)

In [ ]:
m.metrics_names

In [ ]:
m.evaluate(x = image_batch,
               y = mask_batch,
               batch_size = batch_size)

# Making predictions with model

In [ ]:
def predict_one_mask():
    image_batch, mask_batch = next(test_gen)
#     print(type(image_batch))
#     print(type(mask_batch))
    predicted_mask_batch = m.predict(image_batch)
    image = image_batch[0]
#     print(type(predicted_mask_batch))
#     print(type(image))
    print(image.shape)
    predicted_mask = predicted_mask_batch[0].reshape(SIZE, SIZE)
    plt.imshow(image.squeeze())
    plt.imshow(predicted_mask, alpha=0.7)

In [ ]:
predict_one_mask()

In [ ]:
image_batch, mask_batch = next(test_gen)
pred = m.predict(image_batch)
pred = pred > 0.5

In [ ]:
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)

ax = fig.add_subplot(1, 3, 1)
ax.imshow(np.reshape(image_batch[0]*255, (SIZE, SIZE)), cmap='gray')

ax = fig.add_subplot(1, 3, 2)
ax.imshow(np.reshape(mask_batch[0]*255, (SIZE, SIZE)), cmap='gray')

ax = fig.add_subplot(1, 3, 3)
ax.imshow(np.reshape(pred[0]*255, (SIZE, SIZE)), cmap="gray")

In [ ]:
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)

ax = fig.add_subplot(1, 2, 1)
ax.imshow(np.reshape(mask_batch[1]*255, (SIZE, SIZE)), cmap="gray")

ax = fig.add_subplot(1, 2, 2)
ax.imshow(np.reshape(pred[1]*255, (SIZE, SIZE)), cmap="gray")